#### Preparación y Carga de Datos

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Separamos las características (X) de la variable objetivo (y). Las características incluyen aspectos como la elevación, aspecto, pendiente, distancia a cuerpos de agua, entre otros. La variable objetivo es el tipo de cobertura arbórea.

In [9]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from scipy import optimize
%matplotlib inline

# Cargar el dataset
df = pd.read_csv('/content/drive/MyDrive/datasets/covtype.csv')

X = df.drop('Cover_Type', axis=1).values  # .values para convertir a numpy array
y = df['Cover_Type'].values

Implementamos la función sigmoide, que es fundamental para la regresión logística, ya que transforma valores a un rango entre 0 y 1, representando probabilidades.

In [11]:
def calcularSigmoide(z):
    # Calcula la sigmoide de z.
    return 1.0 / (1.0 + np.exp(-z))

La función de costo y gradiente evalúa el rendimiento del modelo y calcula el gradiente de la función de costo; permitiendo la optimización de los parámetros del modelo.


In [12]:
def calcularCosto(theta, X, y, lambda_):
    # Inicializa algunos valores utiles
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = calcularSigmoide(X.dot(theta.T))

    temp = theta
    temp[0] = 0

    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
    # Se aplica regularizacion en la siguiente linea
    grad = grad + (lambda_ / m) * temp

    return J, grad

In [13]:
# valores de prueba para los parámetros theta
theta_t = np.array([-2, -1, 1, 2], dtype=float)

# valores de prueba para las entradas
X_t = np.concatenate((np.ones((5, 1)), np.arange(1, 16).reshape(5, 3, order='F')/10.0), axis=1)
print(X_t)
# valores de testeo para las etiquetas
y_t = np.array([1, 0, 1, 0, 1])

# valores de testeo para el parametro de regularizacion
lambda_t = 3

[[1.  0.1 0.6 1.1]
 [1.  0.2 0.7 1.2]
 [1.  0.3 0.8 1.3]
 [1.  0.4 0.9 1.4]
 [1.  0.5 1.  1.5]]


In [14]:
J, grad = calcularCosto(theta_t, X_t, y_t, lambda_t)

print('Costo         : {:.6f}'.format(J))
print('Costo esperadot: 2.534819')
print('-----------------------')
print('Gradientes:')
print(' [{:.6f}, {:.6f}, {:.6f}, {:.6f}]'.format(*grad))
print('Gradientes esperados:')
print(' [0.146561, -0.548558, 0.724722, 1.398003]');

Costo         : 2.534819
Costo esperadot: 2.534819
-----------------------
Gradientes:
 [0.146561, -0.548558, 0.724722, 1.398003]
Gradientes esperados:
 [0.146561, -0.548558, 0.724722, 1.398003]


Para manejar la clasificación multiclase, utilizamos la estrategia One-vs-All, entrenando un clasificador binario por cada clase. Cada clasificador predice la probabilidad de que una observación pertenezca a su clase en contraposición a todas las demás clases.


Esta función ajusta un modelo para cada tipo de cobertura arbórea, utilizando minimización de la función de costo con regularización. La regularización ayuda a prevenir el sobreajuste.


In [15]:
def oneVsAll(X, y, num_labels, lambda_):
    # Añadir columna de unos a X
    m, n = X.shape
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    all_theta = np.zeros((num_labels, n + 1))

    # Entrenar un clasificador por etiqueta
    for c in np.arange(1, num_labels + 1):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(calcularCosto,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='TNC',
                                options=options)
        all_theta[c-1] = res.x
    return all_theta

# Número de etiquetas
num_labels = np.unique(y).size

# Entrenar el modelo
lambda_ = 0.1
all_theta = oneVsAll(X, y, num_labels, lambda_)


<ipython-input-15-054be94e5663>:11: OptimizeWarning: Unknown solver options: maxiter
  res = optimize.minimize(calcularCosto,
<ipython-input-11-b4b3dba6631b>:4: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))
<ipython-input-12-d22e4f31dd79>:17: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))


Finalmente, realizamos predicciones en el conjunto de datos utilizando los modelos entrenados y evaluamos la precisión del clasificador. La precisión se calcula como el porcentaje de predicciones correctas sobre el total de predicciones realizadas.


In [16]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0]
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(calcularSigmoide(X.dot(all_theta.T)), axis=1) + 1
    return p

# Predecir y calcular la precisión
pred = predictOneVsAll(all_theta, X)
print('Precisión del conjunto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))


Precisión del conjunto de entrenamiento: 71.24%
